In [ ]:
import numpy as np
import cv2

import PIL.Image as Image
#import os

#import matplotlib.pylab as plt

#import tensorflow as tf
#import tensorflow_hub as tensorflow_hub

from tensorflow import keras
#from tensorflow.keras import layers
#from tensorflow.keras.models import Sequential

import pathlib

In [ ]:
confused_dataset_dir = "Engaged/confused"
engaged_dataset_dir = "Engaged/engaged"
frustrated_dataset_dir = "Engaged/frustrated"

Lookingaway_dataset_dir = "Not engaged/Looking Away"
bored_dataset_dir = "Not engaged/bored"
drowsy_dataset_dir = "Not engaged/drowsy"


confused_dataset_dir = pathlib.Path(confused_dataset_dir)
print(confused_dataset_dir)
engaged_dataset_dir = pathlib.Path(engaged_dataset_dir)
print(engaged_dataset_dir)
frustrated_dataset_dir = pathlib.Path(frustrated_dataset_dir)
print(frustrated_dataset_dir)
Lookingaway_dataset_dir = pathlib.Path(Lookingaway_dataset_dir)
print(Lookingaway_dataset_dir)
bored_dataset_dir = pathlib.Path(bored_dataset_dir)
print(bored_dataset_dir)
drowsy_dataset_dir = pathlib.Path(drowsy_dataset_dir)
print(drowsy_dataset_dir)

In [ ]:
list(confused_dataset_dir.glob('*'))[:5]

In [ ]:
image_count = len(list(confused_dataset_dir.glob('*')))
image_count

In [ ]:
confused = list(confused_dataset_dir.glob("*"))
Image.open(str(confused[5])) 

In [ ]:
image_dict = {
    'confused': list(confused_dataset_dir.glob('*')),
    'engaged' : list(engaged_dataset_dir.glob('*')),
    'frustrated': list(frustrated_dataset_dir.glob('*')),
    'Lookingaway' : list(Lookingaway_dataset_dir.glob('*')),
    'bored': list(bored_dataset_dir.glob('*')),
    'drowsy' : list(drowsy_dataset_dir.glob('*')),
}

In [ ]:
labels_dict = {
    'confused': 0,
    'engaged' : 1,
    'frustrated': 2,
    'Lookingaway' : 3,
    'bored': 4,
    'drowsy' : 5,
}

labels_class = ['confused','engaged','frustrated','Lookingaway','bored','drowsy',]

In [ ]:
img = cv2.imread(str(image_dict['engaged'][5]))
img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB) 
img.shape

In [ ]:
X = []
y = []

for label, images in image_dict.items():
  for image in images:
    img = cv2.imread(str(image))
    resized_img = cv2.resize(img,(360,360))
    X.append(resized_img)
    y.append(labels_dict[label])

In [ ]:
X[0]

In [ ]:
X = np.array(X)
y = np.array(y)

In [ ]:
X[0].shape

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [ ]:
X_test.shape

In [ ]:
X_train = X_train / 255.0
X_test = X_test / 255.0

In [ ]:
model = keras.Sequential([
    keras.layers.Conv2D(filters=32,kernel_size=(3,3),activation='relu',input_shape=(360,360,3)),
    keras.layers.MaxPooling2D((2,2)),

    keras.layers.Conv2D(filters=64,kernel_size=(3,3),activation='relu',input_shape=(360,360,3)),
    keras.layers.MaxPooling2D((2,2)),

    keras.layers.Flatten(),
    keras.layers.Dense(64,activation='relu'),
    keras.layers.Dense(6,activation='softmax')
])

In [ ]:
model.compile(optimizer='adam',
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=2)  

In [ ]:
model.evaluate(X_test,y_test)

In [ ]:
predictions = model.predict(X_test)

In [ ]:
predictions.shape

In [ ]:
verify_index = 450

In [ ]:
pred_index = np.argmax(predictions[verify_index])

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(X_test[verify_index])
plt.xlabel(labels_class[y_test[verify_index]])
print("Pridicted state: ",labels_class[pred_index])

In [ ]:
out_img = Image.fromarray(X_test[300],'RGB')
out_img.show()

In [ ]:
history = model.fit(X_train,y_train,epochs = 10 , validation_data = (X_test, y_test))

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(10)

plt.figure(figsize=(15, 15))
plt.subplot(2, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
predictions = model.predict(X_test)

In [ ]:
verify_index = 150

In [ ]:
pred_index = np.argmax(predictions[verify_index])

In [ ]:
plt.imshow(X_test[verify_index])
plt.xlabel(labels_class[y_test[verify_index]])
print("Pridicted state: ",labels_class[pred_index])

In [ ]:
rohit_dataset_dir = "rohit"
rohit_dataset_dir = pathlib.Path(rohit_dataset_dir)

In [ ]:
rohit_images = list(rohit_dataset_dir.glob('*'))
rohit_images

In [ ]:
X_rohit = []

for image in rohit_images:
    img = cv2.imread(str(image))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    resized_img = cv2.resize(img,(360,360))
    X_rohit.append(resized_img)

In [ ]:
X_rohit = np.array(X_rohit)
X_rohit = X_rohit / 255.0

In [ ]:
predictions = model.predict(X_rohit)

In [ ]:
predictions

In [ ]:
rohit_pred_index = np.argmax(predictions[1])

In [ ]:
plt.imshow(X_rohit[1])
#plt.xlabel(labels_class[y_test[2]])
print("Pridicted state: ",labels_class[rohit_pred_index])